<a href="https://colab.research.google.com/github/alzzeaim/pytorch/blob/main/import_MNIST_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [13]:
transform = transforms.ToTensor()

In [14]:
train_data = datasets.MNIST(root='/cnn_data', train=True, download=True, transform=transform)

In [15]:
test_data = datasets.MNIST(root='/cnn_data', train=False, download=True, transform=transform)

In [21]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [22]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [17]:
train_loader =  DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=False)

In [19]:
conv1 = nn.Conv2d(1,6,3,1)
conv2 = nn.Conv2d(6,16,3,1)

In [23]:
for i, (X_train, y_train) in enumerate(train_loader):
    break

In [24]:
X_train

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0.

In [25]:
X_train.shape

torch.Size([10, 1, 28, 28])

In [27]:
x = X_train.view(10,1,28,28)

In [28]:
x = F.relu(conv1(x))

In [30]:
x.shape

torch.Size([10, 6, 26, 26])

In [32]:
x = F.max_pool2d(x,2,2)

In [33]:
x.shape

torch.Size([10, 6, 13, 13])

In [34]:
x = F.relu(conv2(x))

In [35]:
x.shape

torch.Size([10, 16, 11, 11])

In [36]:
x = F.max_pool2d(x,2,2)

In [37]:
x.shape

torch.Size([10, 16, 5, 5])

In [39]:
# model class
class convlutionalNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1,6,3,1)
    self.conv2 = nn.Conv2d(6,16,3,1)
    self.fc1 = nn.Linear(5*5*16, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
      x = F.relu(self.conv1(x))
      x = F.max_pool2d(x,2,2)
      x = F.relu(self.conv2(x))
      x = F.max_pool2d(x,2,2)

      x = x.view(-1, 16*5*5)
      x = F.relu(self.fc1(x))
      x = F.relu(self.fc2(x))
      x = self.fc3(x)
      return F.log_softmax(x, dim=1)

In [43]:
torch.manual_seed(41)
model = convlutionalNet()
model

convlutionalNet(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [44]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [46]:
import time
srart_time = time.time()


epochs = 5
train_losses = []
test_losses = []
train_correct = []
test_correct = []


for i in range(epochs):
  trn_corr = 0
  tst_corr = 0


  for b, (X_train, y_train) in enumerate(train_loader):

    b+=1
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)

    predicted = torch.max(y_pred.data, 1)[1]
    batch_corr = (predicted == y_train).sum()
    trn_corr += batch_corr


    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


    if b%600 == 0:
      print(f'epoch: {i}  batch: {b}  loss: {loss.item()}')

  train_losses.append(loss)
  train_correct.append(trn_corr)
  with torch.no_grad():
    for b, (X_test, y_test) in enumerate(test_loader):
       y_val = model(X_test)
       predicted = torch.max(y_val.data, 1)[1]
       tst_corr += (predicted == y_test).sum()


  loss = criterion(y_val, y_test)
  test_losses.append(loss)
  test_correct.append(tst_corr)






current_time = time.time()
totle = current_time - srart_time
print(f'Training Took :{total/60} minutes')

NotImplementedError: Module [convlutionalNet] is missing the required "forward" function